In [73]:
using CSV, DataFrames

In [74]:
root = dirname(@__FILE__)
inputAIS = joinpath(root, "AI_factor_full_1997-2021.txt")
inputISI = joinpath(root, "IF_factor_full_1997-2021.txt")

"/home/lali/TITAN-ROG-sync/julia/ISI/IF_factor_full_1997-2021.txt"

In [75]:
AIS = CSV.read(inputAIS, DataFrame; header=true)
AIS[8157, [1, 2, 3, 14, 25, 26]]

,Journal,1997,1998,2009,2020,2021
,String31,Float64,Float64,Float64,Float64,Float64
8157,IEEE MAGN LETT,0.789,0.789,0.789,0.409,0.409


In [76]:
ISI = CSV.read(inputISI, DataFrame; header=true)
ISI[8157, [1, 2, 3, 14, 25, 26]]

,Journal,1997,1998,2009,2020,2021
,String31,Float64,Float64,Float64,Float64,Float64
8157,IEEE MAGN LETT,1.692,1.692,1.692,1.549,1.549


In [77]:
root = dirname(@__FILE__)
inputMINE = joinpath(root, "LS_savedrecs_mine.txt")
inputCIT = joinpath(root, "LS_savedrecs.txt")

"/home/lali/TITAN-ROG-sync/julia/ISI/LS_savedrecs.txt"

In [78]:
MINEraw = CSV.read(inputMINE, DataFrame; header=true)
eltype.(eachcol(MINEraw));

┌ Warning: thread = 1 warning: parsed expected 67 columns, but didn't reach end of line around data row: 1. Parsing extra columns and widening final columnset
└ @ CSV /home/lali/.julia/packages/CSV/nofYz/src/file.jl:635


In [79]:
MINE = MINEraw[:, [2, 9, 24, 30, 32, 42, 45, 46, 47, 52, 53, 54, 55]];
eltype.(eachcol(MINE));

In [80]:
MINE[1, [:PY]] .= 2021   # ADV MAT no year!!!!
eltype.(eachcol(MINE));

In [81]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

Pregatim MINE cu noi coloane

In [82]:
rename!(MINE,:"J9" => :"Journal")
insertcols!(MINE, :NA => 0);
insertcols!(MINE, :NEFF => 0);
insertcols!(MINE, :PRIM => 0);
insertcols!(MINE, :AI => 0.0);
insertcols!(MINE, :AINEFF => 0.0);
insertcols!(MINE, :IF => 0.0);
insertcols!(MINE, :IFNA => 0.0);

Gasim numarul de autori si prim autor

In [83]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    MINE[row_index, [:NA]] .= (1+count(";", lucrare[:AU]))
    if occursin("Stoleriu", lucrare[:RP])
        MINE[row_index, [:PRIM]] .= 1
    end
    #println(row_index, MINE[row_index, [:NA]])
    #println(row_index, MINE[row_index, [:PRIM]])
end

In [84]:
#MINE[:, [6, 7, 8, 12, 14, 15, 16, 17]]
MINE.PY = Int64.(MINE.PY)
MINE[ismissing.(MINE[!, :PY]), [1, 6, 7, 8, 12, 14, 15, 16, 17]]

,AU,Journal,PY,VL,AR,NA,NEFF,PRIM,AI
,String,String31?,Int64,Int64?,String7?,Int64,Int64,Int64,Float64


In [85]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

In [86]:
AISsubsect = filter(r -> any(occursin.(Vector(MINE[1, [:Journal]]), r.Journal)), AIS)
#subsect2 = filter(r -> (["ADV MATER"][1] .== r.Journal[1]), AIS)
size(AISsubsect, 1)
#["ADV MATER"][1] .== subsect[1, [:Journal]][1]

10

In [87]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    #lucrare = MINE[1, :]
    if(ismissing(lucrare[:Journal]))
        continue
    end
    for AISrow in eachrow(AIS)
        if( lucrare[:Journal] .== AISrow[:Journal] )
            anString = string(lucrare[:PY])
            #println(AISrow[anString])
            MINE[row_index, [:AI]] .= AISrow[anString]
        end
    end
    for ISIrow in eachrow(ISI)
        if( lucrare[:Journal] .== ISIrow[:Journal] )
            anString = string(lucrare[:PY])
            #println(ISIrow[anString])
            MINE[row_index, [:IF]] .= ISIrow[anString]
            MINE[row_index, [:IFNA]] .= ISIrow[anString]/MINE[row_index, [:NA]][1]
        end
    end
end

In [88]:
root = dirname("/mnt/d/Stoleriu/")
outputMINE = joinpath(root, "MINE_test.csv")
CSV.write(outputMINE, MINE)

"/mnt/d/Stoleriu/MINE_test.csv"